# MLOps. Практическое задание №1 (vo_HW). Мулявин А.А.

## Этапы:

1. Создайте python-скрипт (```data_creation.py```), который создает различные наборы данных, описывающие некий процесс (например, изменение дневной температуры). Таких наборов должно быть несколько, в некоторые данные можно включить аномалии или шумы. Часть наборов данных должна быть сохранена в папке «train», другая часть — в папке «test».

2. Создайте python-скрипт (```model_preprocessing.py```), который выполняет предобработку данных, например с помощью sklearn.preprocessing.StandardScaler.

3. Создайте python-скрипт (```model_preparation.py```), который создает и обучает модель машинного обучения на построенных данных из папки «train».

4. Создайте python-скрипт (```model_testing.py```), проверяющий модель машинного обучения на построенных данных из папки «test».

5. Напишите bash-скрипт (```pipeline.sh```), последовательно запускающий все python-скрипты.

In [41]:
# Импорт библиотек
import os

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [42]:
# Сохранение наборов
def save_dataset(df: pd.DataFrame, name: str, isTest:bool = False) -> None:
    path = "data/train"
    if isTest:
        path = "data/test"
    if not os.path.exists(path):
        os.makedirs(path)
    df.to_csv(f"{path}/{name}.csv", index=False)

## Этап 1. Генерация данных

### 1.1. Настройки генератора

In [191]:
# Константы общие
RANDOM_STATE = 42
SAMPLE_SIZE = 100_000

# Диапазон лет
YEAR_FROM = 2020
YEAR_TO = 2024

# Константы - Стаж работы
WORK_EXPERIENCE_FROM = 0
WORK_EXPERIENCE_TO = 20

# Грейды стажа
WORK_GRADES_DF = pd.DataFrame.from_dict([
    {"name": "Young Padawan", "from": 0, "to": 1, "koef": 0.6, "year_koef": 0.0},
    {"name": "Junior", "from": 1, "to": 3, "koef": 0.8, "year_koef": 0.0},
    {"name": "Middle", "from": 3, "to": 7, "koef": 1.1, "year_koef": 0.05},
    {"name": "Senior", "from": 7, "to": 12, "koef": 1.5, "year_koef": 0.08},
    {"name": "Architect", "from": 12, "to": 100, "koef": 1.9, "year_koef": 0.15},
]) 
WORK_GRADES_DF.columns=["WORK_GRADE_NAME", "WORK_GRADE_YEAR_FROM", "WORK_GRADE_YEAR_TO", "WORK_GRADE_KOEF", "WORK_GRADE_YEAR_KOEF"]

# Модель работы
WORK_MODELS_DF = pd.DataFrame.from_dict([
    {"name": "On-Site", "koef": 1.0},
    {"name": "Remote",  "koef": 0.7},
    {"name": "Hybrid",  "koef": 0.85}
])
WORK_MODELS_DF.columns=["WORK_MODEL_NAME", "WORK_MODEL_KOEF"]

# Тип занятости
EMPLOYMENT_TYPES_DF = pd.DataFrame.from_dict([
    {"name": "Full-Time", "koef": 1.0},
    {"name": "Contract", "koef": 1.2}
])
EMPLOYMENT_TYPES_DF.columns=["EMPLOYMENT_TYPE_NAME", "EMPLOYMENT_TYPE_KOEF"]

# Должности
JOB_TITLES_DF = pd.DataFrame.from_dict([
    {"name": "Data Engineer", "base": 2000.0},
    {"name": "BI Developer", "base": 1750.0},
    {"name": "Developer", "base": 1640.0},
])
JOB_TITLES_DF.columns=["JOB_TITLE_NAME", "JOB_TITLE_BASE"]

# Параметры шума по целевой переменной
SALARY_NOISE_LOW = -0.3
SALARY_NOISE_HIGH = 0.3

# Параметры шума по параметрам
WORK_EXPERIENCE_NOISE_LOW = -1
WORK_EXPERIENCE_NOISE_HIGH = 1

In [192]:
# Настройка рандомизатора
np.random.seed(RANDOM_STATE)

### 1.2. Получение рандомных ключей

In [193]:
# Генерация Диапазона лет
YEAR_KEYS = np.random.randint(
    YEAR_FROM, 
    YEAR_TO, 
    size=SAMPLE_SIZE)
YEAR_KEYS

array([2022, 2023, 2020, ..., 2022, 2023, 2021])

In [194]:
# Генерация Стажа работы
WORK_EXPERIENCE_KEYS = np.random.randint(
    WORK_EXPERIENCE_FROM, 
    WORK_EXPERIENCE_TO, 
    size=SAMPLE_SIZE)
WORK_EXPERIENCE_KEYS

array([ 8,  3,  5, ..., 17,  6,  2])

In [195]:
# Расчет Грейдов
WORK_GRADE_KEYS = np.zeros(
    shape=(SAMPLE_SIZE),
    dtype=int
)

for i in range(SAMPLE_SIZE):
    work_experience: int = WORK_EXPERIENCE_KEYS[ i ]
    EXP_FROM_CL = WORK_GRADES_DF["WORK_GRADE_YEAR_FROM"] <= work_experience
    EXP_TO_CL = WORK_GRADES_DF["WORK_GRADE_YEAR_TO"] >= work_experience
    RESULT_INDEX_LIST = WORK_GRADES_DF.index[EXP_FROM_CL & EXP_TO_CL]
    if len(RESULT_INDEX_LIST) > 0:
        WORK_GRADE_KEYS[ i ] = RESULT_INDEX_LIST[ 0 ]
    else:
        print(f"{work_experience}")
        WORK_GRADE_KEYS[ i ] = np.NAN

WORK_GRADE_KEYS

array([3, 1, 2, ..., 4, 2, 1])

In [196]:
# Генерация Модели работы
WORK_MODEL_KEYS = np.random.randint(
    0, 
    len(WORK_MODELS_DF), 
    size=SAMPLE_SIZE)
WORK_MODEL_KEYS

array([1, 0, 1, ..., 0, 1, 1])

In [197]:
# Генерация Типа занятости
EMPLOYMENT_TYPE_KEYS = np.random.randint(
    0, 
    len(EMPLOYMENT_TYPES_DF), 
    size=SAMPLE_SIZE)
EMPLOYMENT_TYPE_KEYS

array([0, 1, 0, ..., 1, 1, 1])

In [198]:
# Генерация Должности
JOB_TITLE_KEYS = np.random.randint(
    0, 
    len(JOB_TITLES_DF), 
    size=SAMPLE_SIZE)
JOB_TITLE_KEYS

array([2, 1, 0, ..., 2, 1, 1])

### 1.3. Создание датасета

In [199]:
# Создание единого датасета
df = pd.DataFrame(
    data=np.array([JOB_TITLE_KEYS, EMPLOYMENT_TYPE_KEYS, WORK_MODEL_KEYS, WORK_GRADE_KEYS, WORK_EXPERIENCE_KEYS, YEAR_KEYS]).T,
    columns=['JOB_TITLE_KEYS', 'EMPLOYMENT_TYPE_KEYS', 'WORK_MODEL_KEYS', 'WORK_GRADE_KEYS', 'WORK_EXPERIENCE_YEARS', 'YEAR']
)
df

,JOB_TITLE_KEYS,EMPLOYMENT_TYPE_KEYS,WORK_MODEL_KEYS,WORK_GRADE_KEYS,WORK_EXPERIENCE_YEARS,YEAR
0,2,0,1,3,8,2022
1,1,1,0,1,3,2023
2,0,0,1,2,5,2020
3,2,1,2,0,1,2022
4,2,1,1,0,0,2022
...,...,...,...,...,...,...
99995,1,1,0,4,14,2022
99996,2,1,1,3,11,2023
99997,2,1,0,4,17,2022
99998,1,1,1,2,6,2023


In [200]:
# Соединение со справочником JOB_TITLES
full_df = df.merge(JOB_TITLES_DF, left_on="JOB_TITLE_KEYS", right_index=True) \
    .merge(EMPLOYMENT_TYPES_DF, left_on="EMPLOYMENT_TYPE_KEYS", right_index=True) \
    .merge(WORK_MODELS_DF, left_on="WORK_MODEL_KEYS", right_index=True) \
    .merge(WORK_GRADES_DF, left_on="WORK_GRADE_KEYS", right_index=True) \
    .drop(columns=["JOB_TITLE_KEYS", "EMPLOYMENT_TYPE_KEYS", "WORK_MODEL_KEYS", "WORK_GRADE_KEYS"])
full_df.sample(10)

,WORK_EXPERIENCE_YEARS,YEAR,JOB_TITLE_NAME,JOB_TITLE_BASE,EMPLOYMENT_TYPE_NAME,EMPLOYMENT_TYPE_KOEF,WORK_MODEL_NAME,WORK_MODEL_KOEF,WORK_GRADE_NAME,WORK_GRADE_YEAR_FROM,WORK_GRADE_YEAR_TO,WORK_GRADE_KOEF,WORK_GRADE_YEAR_KOEF
32948,13,2021,BI Developer,1750.0,Contract,1.2,Hybrid,0.85,Architect,12,100,1.9,0.15
89707,1,2022,Developer,1640.0,Full-Time,1.0,Hybrid,0.85,Young Padawan,0,1,0.6,0.00
24738,15,2023,BI Developer,1750.0,Full-Time,1.0,Hybrid,0.85,Architect,12,100,1.9,0.15
99098,10,2022,BI Developer,1750.0,Full-Time,1.0,Hybrid,0.85,Senior,7,12,1.5,0.08
53056,11,2020,Developer,1640.0,Full-Time,1.0,On-Site,1.00,Senior,7,12,1.5,0.08
46934,0,2020,Developer,1640.0,Full-Time,1.0,Remote,0.70,Young Padawan,0,1,0.6,0.00
27559,4,2022,Developer,1640.0,Contract,1.2,Hybrid,0.85,Middle,3,7,1.1,0.05
59336,2,2023,BI Developer,1750.0,Contract,1.2,On-Site,1.00,Junior,1,3,0.8,0.00
52207,9,2022,Data Engineer,2000.0,Contract,1.2,Remote,0.70,Senior,7,12,1.5,0.08
45420,15,2021,Data Engineer,2000.0,Full-Time,1.0,Hybrid,0.85,Architect,12,100,1.9,0.15


In [201]:
# Расчет ЗП
def calc_salary(row) -> float:
    salary = row["JOB_TITLE_BASE"]
    salary = salary * row["EMPLOYMENT_TYPE_KOEF"]
    salary = salary * row["WORK_MODEL_KOEF"]    # Да, удаленщики получают чуть ниже офисников
    salary = salary * row["WORK_GRADE_KOEF"]
    salary = salary + (salary * row["WORK_GRADE_YEAR_KOEF"]) * (row["WORK_EXPERIENCE_YEARS"] - row["WORK_GRADE_YEAR_FROM"])
    return salary

full_df['SALARY'] = full_df.apply(calc_salary, axis=1)

In [202]:
# Удаление служебных полей 
# В теории мы можем знать коэффициенты 
# расчета ЗП, но сейчас они нам не нужны
full_df.drop(['JOB_TITLE_BASE', 'EMPLOYMENT_TYPE_KOEF',
              'WORK_MODEL_KOEF', 'WORK_GRADE_YEAR_FROM',
              'WORK_GRADE_YEAR_TO', 'WORK_GRADE_KOEF',
              'WORK_GRADE_YEAR_KOEF'],
              axis=1,
              inplace=True)

In [203]:
# Вывод финального результата
full_df.sample(10)

,WORK_EXPERIENCE_YEARS,YEAR,JOB_TITLE_NAME,EMPLOYMENT_TYPE_NAME,WORK_MODEL_NAME,WORK_GRADE_NAME,SALARY
69868,18,2022,BI Developer,Full-Time,On-Site,Architect,6317.500
85959,18,2020,Data Engineer,Contract,Hybrid,Architect,7364.400
63403,18,2020,BI Developer,Contract,Hybrid,Architect,6443.850
64721,2,2023,BI Developer,Contract,Hybrid,Junior,1428.000
12875,13,2021,BI Developer,Full-Time,Remote,Architect,2676.625
7957,3,2023,Developer,Contract,On-Site,Junior,1574.400
36186,2,2020,Data Engineer,Contract,Remote,Junior,1344.000
12784,19,2022,BI Developer,Contract,Remote,Architect,5725.650
6773,10,2022,Data Engineer,Contract,Remote,Senior,3124.800
39113,10,2021,Data Engineer,Contract,On-Site,Senior,4464.000


### 1.4. Формирование набора 1

Данные полностью сгенерированные, без шумов и выбросов. Ожидаются высокие показатели модели, так как зависимость данных подчиняется простой формуле.

In [204]:
# Разбиение на учебную и тестовую выборки
data1_train_df, data1_test_df = train_test_split(
    full_df, 
    test_size=0.3, 
    random_state=RANDOM_STATE)

In [205]:
data1_train_df.sample(5)

,WORK_EXPERIENCE_YEARS,YEAR,JOB_TITLE_NAME,EMPLOYMENT_TYPE_NAME,WORK_MODEL_NAME,WORK_GRADE_NAME,SALARY
67021,14,2023,BI Developer,Full-Time,Hybrid,Architect,3674.125
69295,18,2020,Developer,Full-Time,Remote,Architect,4144.280
92866,15,2023,Developer,Full-Time,On-Site,Architect,4518.200
71264,19,2021,Developer,Full-Time,Remote,Architect,4471.460
27510,6,2022,Developer,Contract,Hybrid,Middle,2116.092


In [206]:
data1_test_df.sample(5)

,WORK_EXPERIENCE_YEARS,YEAR,JOB_TITLE_NAME,EMPLOYMENT_TYPE_NAME,WORK_MODEL_NAME,WORK_GRADE_NAME,SALARY
76813,10,2022,Developer,Full-Time,Hybrid,Senior,2592.84
9276,7,2021,Data Engineer,Contract,On-Site,Middle,3168.00
99722,9,2020,BI Developer,Full-Time,Hybrid,Senior,2588.25
12925,9,2022,BI Developer,Contract,Remote,Senior,2557.80
21939,0,2021,Data Engineer,Full-Time,Hybrid,Young Padawan,1020.00


In [207]:
# Сохранение в папки
save_dataset(data1_train_df, "data1")
save_dataset(data1_test_df, "data1", isTest=True)

### 1.5. Формирование набора 2

Добавление шума в целевую переменную.

In [208]:
# Копирование исходного датасета
full_2_df = full_df.copy(True)

In [209]:
# Генерация шума для целевой переменной в диапазоне от 0.3 до -0.3
salary_noise_koef = np.random.uniform(
    low=SALARY_NOISE_LOW, 
    high=SALARY_NOISE_HIGH, 
    size=SAMPLE_SIZE)
salary_noise_koef

array([-0.05450853,  0.16140049, -0.10666283, ..., -0.11285634,
        0.11354892,  0.12211903])

In [210]:
# Добавление шума в отдельный столбец
full_2_df["SALARY"] = full_2_df["SALARY"] + full_2_df["SALARY"] * salary_noise_koef
full_2_df.sample(10)

,WORK_EXPERIENCE_YEARS,YEAR,JOB_TITLE_NAME,EMPLOYMENT_TYPE_NAME,WORK_MODEL_NAME,WORK_GRADE_NAME,SALARY
85343,18,2021,Developer,Full-Time,On-Site,Architect,4859.368395
71255,14,2020,BI Developer,Full-Time,Remote,Architect,3580.461133
51135,13,2020,Developer,Contract,On-Site,Architect,4196.068548
16796,8,2023,Developer,Full-Time,Remote,Senior,2252.106714
71686,14,2021,BI Developer,Full-Time,On-Site,Architect,5594.320374
14731,13,2020,Developer,Contract,Remote,Architect,2764.761196
58507,15,2022,Data Engineer,Contract,Hybrid,Architect,5272.674183
81381,12,2020,Data Engineer,Contract,Remote,Senior,3142.058933
61006,16,2020,BI Developer,Contract,Remote,Architect,3625.306010
4933,18,2023,Developer,Contract,Remote,Architect,5291.740421


In [211]:
# Разбиение на учебную и тестовую выборки
data2_train_df, data2_test_df = train_test_split(
    full_2_df, 
    test_size=0.3, 
    random_state=RANDOM_STATE)

In [212]:
data2_train_df.sample(5)

,WORK_EXPERIENCE_YEARS,YEAR,JOB_TITLE_NAME,EMPLOYMENT_TYPE_NAME,WORK_MODEL_NAME,WORK_GRADE_NAME,SALARY
50384,8,2022,BI Developer,Contract,On-Site,Senior,3425.673651
61264,10,2023,Developer,Full-Time,On-Site,Senior,2585.423739
80955,15,2021,Data Engineer,Full-Time,Remote,Architect,3517.817027
71299,7,2021,BI Developer,Contract,Remote,Middle,2456.630121
37349,8,2022,Data Engineer,Contract,On-Site,Senior,4497.704472


In [213]:
data2_test_df.sample(5)

,WORK_EXPERIENCE_YEARS,YEAR,JOB_TITLE_NAME,EMPLOYMENT_TYPE_NAME,WORK_MODEL_NAME,WORK_GRADE_NAME,SALARY
71535,14,2022,Developer,Contract,Remote,Architect,2872.117193
64787,11,2023,Data Engineer,Full-Time,Hybrid,Senior,3716.239900
53399,5,2022,Data Engineer,Contract,On-Site,Middle,3277.951839
76715,2,2020,BI Developer,Full-Time,Hybrid,Junior,1453.089233
72543,16,2022,Data Engineer,Contract,Hybrid,Architect,7005.550358


In [214]:
# Сохранение в папки
save_dataset(data2_train_df, "data2")
save_dataset(data2_test_df, "data2", isTest=True)

### 1.6. Формирование набора 3

Добавление шума к стажу работы и выбросов в целевую переменную.

In [215]:
# Копирование исходного датасета
full_3_df = full_df.copy(True)

In [216]:
# Генерация шума для стажа работы (кто то приверает на 1 или 2 года)
work_experience_noise_koef = np.random.choice([0, 1, 2], SAMPLE_SIZE, p=[0.9, 0.08, 0.02])
work_experience_noise_koef

array([0, 0, 0, ..., 0, 2, 0])

In [217]:
print(f"Результат распределения: {dict(zip(*np.unique(work_experience_noise_koef, return_counts=True)))}")

Результат распределения: {0: 90040, 1: 7996, 2: 1964}


In [218]:
# Добавление шума в отдельный столбец
full_3_df["WORK_EXPERIENCE_YEARS"] = full_3_df["WORK_EXPERIENCE_YEARS"] + work_experience_noise_koef
full_3_df.sample(10)

,WORK_EXPERIENCE_YEARS,YEAR,JOB_TITLE_NAME,EMPLOYMENT_TYPE_NAME,WORK_MODEL_NAME,WORK_GRADE_NAME,SALARY
84273,1,2021,Data Engineer,Full-Time,On-Site,Young Padawan,1200.000
20756,12,2021,BI Developer,Full-Time,Remote,Senior,2572.500
34072,13,2021,Developer,Contract,Hybrid,Architect,3655.068
37326,8,2021,Data Engineer,Contract,On-Site,Senior,3888.000
34126,4,2023,BI Developer,Contract,On-Site,Middle,2425.500
57624,15,2021,Data Engineer,Full-Time,Hybrid,Architect,4683.500
36313,0,2020,Developer,Full-Time,Hybrid,Young Padawan,836.400
88716,15,2021,Data Engineer,Full-Time,Hybrid,Architect,4683.500
31363,4,2022,Developer,Contract,Hybrid,Middle,1932.084
16291,10,2021,Developer,Full-Time,Remote,Senior,2135.280


In [219]:
# Генерация выбросов по ЗП (выбросы почти в разы)
outliers_koef = np.random.choice([-2.5, -2, 0, 2, 3], SAMPLE_SIZE, p=[0.01, 0.04, 0.9, 0.04, 0.01])
print(f"Результат распределения: {dict(zip(*np.unique(outliers_koef, return_counts=True)))}")

Результат распределения: {-2.5: 1003, -2.0: 4001, 0.0: 90045, 2.0: 3950, 3.0: 1001}


In [220]:
# Применение выбросов на данные
full_3_df["SALARY"] = full_3_df["SALARY"] + full_3_df["SALARY"] * outliers_koef / 10
full_3_df.sample(10)

,WORK_EXPERIENCE_YEARS,YEAR,JOB_TITLE_NAME,EMPLOYMENT_TYPE_NAME,WORK_MODEL_NAME,WORK_GRADE_NAME,SALARY
56686,6,2023,Data Engineer,Full-Time,On-Site,Middle,2530.0000
15543,5,2021,BI Developer,Full-Time,On-Site,Middle,2117.5000
80157,12,2020,Data Engineer,Full-Time,Remote,Senior,2940.0000
33388,20,2023,BI Developer,Contract,Remote,Architect,5725.6500
89016,10,2023,Data Engineer,Full-Time,Hybrid,Senior,3162.0000
90639,11,2021,BI Developer,Contract,On-Site,Senior,4158.0000
9586,6,2022,Developer,Contract,Hybrid,Middle,2116.0920
16751,13,2021,BI Developer,Full-Time,On-Site,Architect,3823.7500
23445,1,2023,Developer,Contract,Hybrid,Young Padawan,1003.6800
62219,20,2022,BI Developer,Full-Time,Hybrid,Architect,5793.8125


In [221]:
# Разбиение на учебную и тестовую выборки
data3_train_df, data3_test_df = train_test_split(
    full_3_df, 
    test_size=0.3, 
    random_state=RANDOM_STATE)

In [222]:
data3_train_df.sample(5)

,WORK_EXPERIENCE_YEARS,YEAR,JOB_TITLE_NAME,EMPLOYMENT_TYPE_NAME,WORK_MODEL_NAME,WORK_GRADE_NAME,SALARY
83746,0,2020,Data Engineer,Full-Time,Hybrid,Young Padawan,1020.0000
24664,12,2021,Data Engineer,Full-Time,Remote,Senior,2940.0000
41819,12,2022,BI Developer,Contract,On-Site,Senior,4410.0000
43903,9,2022,Data Engineer,Contract,On-Site,Senior,4176.0000
43220,14,2020,BI Developer,Full-Time,Hybrid,Architect,3250.1875


In [223]:
data3_test_df.sample(5)

,WORK_EXPERIENCE_YEARS,YEAR,JOB_TITLE_NAME,EMPLOYMENT_TYPE_NAME,WORK_MODEL_NAME,WORK_GRADE_NAME,SALARY
31735,5,2020,Data Engineer,Contract,Remote,Middle,2032.80
29451,4,2023,Data Engineer,Full-Time,Remote,Middle,1617.00
93348,7,2023,Developer,Full-Time,Hybrid,Middle,1840.08
23850,4,2021,Data Engineer,Full-Time,Remote,Middle,1617.00
66233,8,2021,Data Engineer,Full-Time,Remote,Middle,1771.00


In [224]:
# Сохранение в папки
save_dataset(data3_train_df, "data3")
save_dataset(data3_test_df, "data3", isTest=True)

## Этап 2. Предобработка данных

In [225]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [226]:
# Функция предварительной обработки данных
def preprocess_data(train_df: pd.DataFrame, test_df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    # В учебных целях мудрить не будем, сделаем базовые преобразования
    # - One-Hot кодирование для признаков с малым количеством значений:
    #   + JOB_TITLE_NAME	
    #   + EMPLOYMENT_TYPE_NAME	
    #   + WORK_MODEL_NAME	
    #   + WORK_GRADE_NAME
    # - Стандартизацию для признаков:
    #   + WORK_EXPERIENCE_YEARS
    #   + YEAR
    #   + SALARY


    # Перед предварительнойо бработкой надо слить датасеты,
    # иначе может не корректно просчитаться коэффициент
    # ИМХО: Делитьн на наборы надо именно тут!
    full_df = pd.concat([train_df, test_df]) 
 
 
    one_hot_cols: list = ['JOB_TITLE_NAME', 'EMPLOYMENT_TYPE_NAME', 'WORK_MODEL_NAME', 'WORK_GRADE_NAME']
    stand_scaler_cols: list = ['WORK_EXPERIENCE_YEARS', 'YEAR', 'SALARY']

    # Создание pipeline для One-Hot кодирования
    pipe_one_hot = Pipeline([
        ('encoder', OneHotEncoder(
            drop='if_binary', 
            handle_unknown='ignore', 
            sparse_output=False
        ))
    ])

    # Создание pipeline для стандартизации
    pipe_stand = Pipeline([
        ('scaler', StandardScaler())
    ])


    # Создание трансформера колонок данных (далее в этом контексте "трансформер")
    preprocessors = ColumnTransformer(transformers=[
        ('stand_cols', pipe_stand, stand_scaler_cols),
        ('one_hot_cols', pipe_one_hot, one_hot_cols)
    ])
    # Обучение трансформера
    preprocessors.fit(full_df)

    # Соберем имена колонок данных после трансформаций
    trans_cols_list = []

    # Колонки при кодировании числовых признаков не изменились
    trans_cols_list.extend(stand_scaler_cols)
    
    # Колонки при One-Hot кодировании добавляются с новыми именами
    for trans in preprocessors.transformers_:
        if trans[0] not in ['one_hot_cols']:
            continue
        pipe: Pipeline = trans[1]
        if 'encoder' in pipe.named_steps:
            trans_cols_list.extend(list(pipe.get_feature_names_out()))


    # Трансформация данных
    copy_train_df = pd.DataFrame(preprocessors.transform(train_df.copy(deep=True)),
                                 columns=trans_cols_list)
    copy_test_df = pd.DataFrame(preprocessors.transform(test_df.copy(deep=True)),
                                columns=trans_cols_list)

    return copy_train_df, copy_test_df

In [227]:
# Чтение данных из наборов данных
train_file_list = [name for name in os.listdir('./data/train/') if name.endswith(".csv") and '_clear' not in name]

for data_file in train_file_list:
    print(f'-> Обработка файлов: {data_file}')
    if not os.path.exists('./data/test/' + data_file):
        print('--> Связанный файл с валидационными данными не найден')
        continue
    data_train_df = pd.read_csv(f"./data/train/{data_file}")
    data_test_df = pd.read_csv(f"./data/test/{data_file}")

    data_train_df, data_test_df = preprocess_data(data_train_df, data_test_df)

    save_dataset(data_train_df,  data_file.replace(".csv", "_clear", 1))
    save_dataset(data_test_df,  data_file.replace(".csv", "_clear", 1), isTest=True)

-> Обработка файлов: data1.csv
-> Обработка файлов: data2.csv
-> Обработка файлов: data3.csv


## Этап 3. Обучение модели

In [229]:
from sklearn.linear_model import LinearRegression

from joblib import dump

In [232]:
# Обучение модели и сохранение на диск
def train_model(train_df: pd.DataFrame, filename: str) -> None:
    
    # Создание модели логистической регрессии
    model = LinearRegression()

    # Разделение данных на параметры и целевую переменную
    X = train_df.drop(['SALARY'], axis=1)
    y  = train_df['SALARY']

    # Обучение модели
    model.fit(X, y)

    # Сохранение модели
    if not os.path.exists('./models/'):
        os.makedirs('./models/')
    dump(model, f'./models/{filename.replace(".csv", ".joblib", 1)}')

In [233]:
# Чтение данных из наборов данных (только обработанные)
train_file_list = [name for name in os.listdir('./data/train/') \
                   if name.endswith(".csv") and '_clear' in name]

for data_file in train_file_list:
    print(f'-> Обработка файла: {data_file}')
    data_train_df = pd.read_csv(f"./data/train/{data_file}")

    train_model(data_train_df, data_file.replace("_clear",  "",  1))

-> Обработка файла: data1_clear.csv
-> Обработка файла: data2_clear.csv
-> Обработка файла: data3_clear.csv


## Этап 4. Проверка модели

In [234]:
from joblib import load
from sklearn.metrics import r2_score

In [235]:
# Тест модели
def model_test(test_df: pd.DataFrame, filename: str) -> float:
    
    # Поиск и загрузка подходящей модели
    model = load(f'./models/{filename.replace(".csv", ".joblib", 1)}')
    if model is None:
        # Это проблема, серьезная проблема
        raise RuntimeError(f"Не найдена модель для {filename}")
    
    # Разделение данных на параметры и целевую переменную
    X = test_df.drop(['SALARY'], axis=1)
    y  = test_df['SALARY']

    # Обучение модели
    y_pred = model.predict(X)

    # Определение метрики
    return r2_score(y, y_pred)

In [237]:
# Чтение данных из наборов данных (только тестовые обработанные)
test_file_list = [name for name in os.listdir('./data/test/') \
                   if name.endswith(".csv") and '_clear' in name]

for data_file in test_file_list:
    print(f'-> Обработка файла: {data_file}')
    data_test_df = pd.read_csv(f"./data/test/{data_file}")

    r2 = model_test(data_test_df, data_file.replace("_clear",  "",  1))
    print(f'--> Оценка модели файла ({data_file.replace("_clear",  "",  1)}):  {r2:.3f}')   

-> Обработка файла: data1_clear.csv
--> Оценка модели файла (data1.csv):  0.951
-> Обработка файла: data2_clear.csv
--> Оценка модели файла (data2.csv):  0.845
-> Обработка файла: data3_clear.csv
--> Оценка модели файла (data3.csv):  0.926


## Дополнение: sh-скрипт

In [38]:
# #!/bin/bash

# Установка зависимостей 
echo "Установка зависемостей"
pip3 install -r ./requirements.txt

# Этап 1: Генерация данных
echo "Генерация данных"
python3 ./src/data_creation.py

# Этап 2: Предобработка данных для модели
echo "Предобработка данных для модели"
python3 ./src/model_preprocessing.py

# Этап 3: Обучение модели
echo "Обучение модели"
python3 ./src/model_preparation.py

# Этап 4: Тест модели и оценка метрик
echo "Тест модели и оценка метрик"
python3 ./src/model_testing.py